### Implement the Hodgkin-Huxley model

## $c_m \frac{\partial V}{\partial t} = \frac{1}{2ar_L} \frac{\partial}{\partial x} (a^2 \frac{\partial V}{\partial x}) - i_m + i_e$

where:

## $i_m = \bar{g}_L (V-E_L) + \bar{g}_{Na} m^3 h (V-E_{Na}) + \bar{g}_{K} n^4 (V-E_{K})$

and gating variables:

## $\frac{dx}{dt} = \alpha_x (V) (1-x) - \beta_x (V) n$

## $\alpha_n (V) = \frac{0.01 (10-V)}{exp \ (\frac{10-V}{10}) - 1} \quad \alpha_m (V) = \frac{0.1(25-V)}{exp \ (\frac{25-V}{10})-1} \quad \alpha_h (V) = 0.07 \ exp \ (\frac{-V}{20})$

## $\beta_n (V) = 0.125 \ exp \ (\frac{-V}{80}) \quad \beta_m (V) = 4 \ exp \ (\frac{-V}{18}) \quad \beta_h (V) = \frac{1}{exp \ (\frac{30-V}{10}) + 1}$

# -------------------------------------------------------------------------------

In [8]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
def C_N(dt, M, Cm, g, A, gm, I, Ie, V):
    """This function calculates dV using the Crank-Nicholson method
    
    dt = timestep, in seconds
    M = number of compartments
    Cm = membrane capacitance, in F
    g = resistive coupling, in S
    A = compartment surface area
    gm = sum of gi
    I = sum of gi * Ei
    Ie = current input
    V = potential
    
    Function uses the method described in Chapter 6.6B from Dayan and Abbott, 2005
    returns one variable, V_new
    """
    z = 0.5 # Crank-Nicholson
    
    ### Helper variables ###
    b = np.zeros(M)
    c = np.zeros(M)
    d = np.zeros(M)
    f = np.zeros(M)
    
    b[1:M] = g * z * dt / Cm[1:M]
    d[0:M-1] = g * z * dt / Cm[0:M-1]
    c = -gm*z*dt/Cm - b - d
    f = (I + Ie/A)/Cm * z * dt + c * V
    for i in range(M-1):
        f[i+1] += b[i+1] * V[i]
        f[i] += d[i] * V[i+1]
    f = f*2 # getting rid of z
    
    ### Forward prop ###
    c1 = np.zeros(M)
    f1 = np.zeros(M)
    
    c1[0] = c[0]
    f1[0] = f[0]
    
    for i in range(M-1):
        c1[i+1] = c[i+1] + b[i+1] * d[i] / (1-c1[i])
        f1[i+1] = f[i+1] + b[i+1] * f1[i] / (1-c1[i])
        
    ### Backprop ###
    dV = np.zeros(M)
    
    dV[M-1] = f1[M-1] / (1-c1[M-1])
    
    for i in range(M-2, -1, -1):
        dV[i] = (d[i] * dV[i+1] + f1[i]) / (1-c1[i])
        
    return V + dV

In [10]:
def HHM(I_na,I_k,I_leak,I_e,C_m):
    dvdt = (1/C_m) * (-(I_na + I_k + I_leak) + I_e)
    return dvdt

def kinetics(x,alpha,beta):
    dxdt = alpha * (1 - x) - beta * x
    return dxdt

In [11]:
def current_k(V,n,g_k,E_k):
    current_k = g_k * (n**4) * (V - E_k)
    return current_k

def current_na(V,m,h,g_na,E_na):
    current_na = g_na * (m**3) * h * (V - E_na)
    return current_na

def conductance_k()

def alpha_n(V):
    alpha = 0.01 * (V + 55) / (1 - np.exp(-0.1 * (V + 55)))
    return alpha

def alpha_m(V):
    alpha = 0.1 * (V + 40) / (1 - exp(-0.1 * (V + 40)))
    return alpha

def alpha_h(V):
    alpha = 0.07 * exp(-0.05 * (V + 65))
    return alpha

def beta_n(V):
    beta = 0.125 * np.exp(-0.0125 * (V+65))
    return beta

def beta_m(V):
    beta = 4 * exp(-0.0556 * (V + 65))
    return beta

def beta_h(V):
    beta = 1 / (1 + exp(-0.1 * (V + 35)))
    return beta

In [14]:
def HHM_combined(initial, t, C_m, I_e, E_na, E_k, E_leak, g_na, g_k, g_leak): 
    
    V,n,m,h = initial #initial values
    
    #calculate potassium current
    kinetics_n = kinetics(n,alpha_n(V),beta_n(V))
    I_k = current_k(V,n,g_k,E_k)
    
    #calculate sodium current
    kinetics_m = kinetics(m,alpha_m(V),beta_m(V))
    kinetics_h = kinetics(h,alpha_h(V),beta_h(V))
    I_na = current_na(V,m,h,g_na,E_na)
    
    #calculate leak current
    I_leak = current_leak(V,g_leak,E_leak)
    
    #calculate voltage 
    voltage = HHM(I_na,I_k,I_leak,I_e,C_m)
    
    return [voltage,kinetics_n,kinetics_m,kinetics_h]

In [21]:
#initial conditions
V0 = -0.065 #(V)
m0 = 0.0529
h0 = 0.5961
n0 = 0.3177
#parameters
C_m = 0.01 #(F)
L = 1e-6 #(m)
a = 238e-6 #(m)
r_L = 35.4e-3 #(ohm m)
g = a / (2 * r_L * L * L) #(S)

#time steps
tmax = 1e-1 # s
dt = 1e-3 # s
N = int(tmax / dt) + 1
t = np.linspace(0, tmax, N)

NameError: name 'gk' is not defined

In [20]:
for i in range(5):
    gm = gk + gna + gl
    solution = C_N(dt=dt, M=1, Cm=C_m, g=g, A=a, gm=gm, I=I, Ie=Ie[:, i], V=V[:, i])

NameError: name 'gm' is not defined

In [ ]:
#plot the solution to the HHM
I_e = 0 #(A)
initial = [V0,n0,m0,h0]
parameters = (C_m, I_e, E_na, E_k, E_leak, g_na, g_k, g_leak, I_following)

plt.plot(t,solution[:,0])
plt.title('Membrane Voltage with $I_e = 0$')
plt.ylabel('voltage [mV]')
plt.xlabel('time [t]')
plt.grid()

#fix offset
ax = plt.gca()
ax.ticklabel_format(useOffset=False)
plt.show()